In [2]:
import gym
from gym import spaces
import numpy as np
from scipy.stats import norm

class OptionHedgingEnv(gym.Env):
    def __init__(self, days=30, S0=500, K=500, sigma=0.2, r=0.01, short_calls=10, hedge_cost_coeff=0.01):
        super(OptionHedgingEnv, self).__init__()
        self.days = days
        self.S0 = S0
        self.K = K
        self.sigma = sigma
        self.r = r
        self.short_calls = short_calls
        self.dt = 1 / 365
        self.hedge_cost_coeff = hedge_cost_coeff
        
        self.action_space = spaces.Box(low=-1, high=1, shape=(1,), dtype=np.float32)  # hedge ratio
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(6,), dtype=np.float32)

    def reset(self):
        self.day = 0
        self.S = self.S0
        self.hedge_position = 0
        self.prev_hedge = 0
        self.total_pnl = 0

        self.daily_returns = np.random.normal(loc=0, scale=self.sigma * np.sqrt(self.dt), size=self.days)
        self.state = self._get_state()
        return self.state

    def step(self, action):
        hedge_ratio = float(np.clip(action[0], -1, 1))
        self.prev_hedge = self.hedge_position
        self.hedge_position = hedge_ratio * self.short_calls

        dS = self.S * (np.exp(self.daily_returns[self.day]) - 1)
        self.S += dS
        self.day += 1
        T = (self.days - self.day) / 365

        # Greeks
        delta = -self.short_calls * self._call_delta(self.S, self.K, T, self.r, self.sigma)
        d_portfolio = delta * dS
        d_hedge = self.hedge_position * dS
        hedge_cost = self.hedge_cost_coeff * abs(self.hedge_position - self.prev_hedge)

        pnl = d_portfolio + d_hedge
        reward = - (pnl ** 2) - hedge_cost

        self.total_pnl += pnl
        done = self.day >= self.days
        self.state = self._get_state()
        return self.state, reward, done, {}

    def _get_state(self):
        T = (self.days - self.day) / 365
        delta = -self.short_calls * self._call_delta(self.S, self.K, T, self.r, self.sigma)
        gamma = -self.short_calls * self._call_gamma(self.S, self.K, T, self.r, self.sigma)
        vega = -self.short_calls * self._call_vega(self.S, self.K, T, self.r, self.sigma)
        return np.array([self.S, T, self.sigma, delta, gamma, vega], dtype=np.float32)

    def _call_delta(self, S, K, T, r, sigma):
        d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
        return norm.cdf(d1)

    def _call_gamma(self, S, K, T, r, sigma):
        d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
        return norm.pdf(d1) / (S * sigma * np.sqrt(T))

    def _call_vega(self, S, K, T, r, sigma):
        d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
        return S * norm.pdf(d1) * np.sqrt(T) / 100


In [4]:
env = OptionHedgingEnv()
obs = env.reset()

done = False
while not done:
    action = env.action_space.sample()  # random hedge
    obs, reward, done, info = env.step(action)
    print(f"Reward: {reward:.4f}, Obs: {obs}")

Reward: -138.7700, Obs: [ 5.0291513e+02  7.9452053e-02  2.0000000e-01 -5.5780277e+00
 -1.3923295e-01 -5.5958562e+00]
Reward: -1853.0996, Obs: [ 5.0724255e+02  7.6712325e-02  2.0000000e-01 -6.1835475e+00
 -1.3568655e-01 -5.3562808e+00]
Reward: -3.1027, Obs: [ 5.0903152e+02  7.3972605e-02  2.0000000e-01 -6.4427180e+00
 -1.3455145e-01 -5.1579685e+00]
Reward: -1715.6095, Obs: [ 5.14248779e+02  7.12328777e-02  2.00000003e-01 -7.14452887e+00
 -1.23792335e-01 -4.66391659e+00]
Reward: -256.0455, Obs: [ 5.1161649e+02  6.8493150e-02  2.0000000e-01 -6.8369069e+00
 -1.3288870e-01 -4.7649055e+00]
Reward: -4002.6726, Obs: [ 5.1605212e+02  6.5753423e-02  2.0000000e-01 -7.4364457e+00
 -1.2166708e-01 -4.2609706e+00]
Reward: -4.7963, Obs: [ 5.1621930e+02  6.3013695e-02  2.0000000e-01 -7.4969101e+00
 -1.2269438e-01 -4.1205778e+00]
Reward: -15.3493, Obs: [ 5.1581738e+02  6.0273971e-02  2.0000000e-01 -7.4892726e+00
 -1.2575239e-01 -4.0333695e+00]
Reward: -6783.2179, Obs: [ 5.2308612e+02  5.7534248e-02  2.0

C:\Users\Manas\AppData\Local\Temp\ipykernel_46540\351937793.py:64: RuntimeWarning: divide by zero encountered in scalar divide
  d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
C:\Users\Manas\AppData\Local\Temp\ipykernel_46540\351937793.py:68: RuntimeWarning: divide by zero encountered in scalar divide
  d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
C:\Users\Manas\AppData\Local\Temp\ipykernel_46540\351937793.py:69: RuntimeWarning: invalid value encountered in scalar divide
  return norm.pdf(d1) / (S * sigma * np.sqrt(T))
C:\Users\Manas\AppData\Local\Temp\ipykernel_46540\351937793.py:72: RuntimeWarning: divide by zero encountered in scalar divide
  d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
